### 1.  OBJETIVO DO CÓDIGO

Calcular automaticamente quanto você precisa repor de cada produto em cada loja, semana a semana, para nunca faltar estoque.

In [34]:
#ativar venv --> .venv\Scripts\activate

from pathlib import Path
import pandas as pd
import numpy as np


DATA_DIR = Path("data")
SAIDA_DIR = Path("outputs")
SAIDA_DIR.mkdir(exist_ok=True)

ARQUIVO_VENDAS = "forecast-vendas.xlsx"
ABA_VENDAS = "Sheet1"
ARQUIVO_ESTOQUE = DATA_DIR / "estoque-total.xlsx"
ARQUIVO_CARTEIRA = DATA_DIR / "carteira.xlsx"

SEMANA_INICIO = "W42_25"
HORIZONTE_SEMANAS = 24 # Usa todas as semanas disponiveis
SEMANAS_ALVO = 12  # Janela de cobertura (~3 meses)
FATOR_DEFALCACAO_ALVO = 0.90  # Aplica deflacao de 10% sobre o alvo


In [35]:
def _ordenar_semana(rotulo):
    """Retorna tupla (ano, semana) para permitir ordenação cronológica."""
    semana, ano = rotulo.split("_") #"W39_25" → (25, 39)
    return int(ano), int(semana[1:]) #(25, 39) → (25, 39)

In [36]:
lista_ra = [
    3, 5, 7, 9, 10, 15, 18, 19, 20, 21, 22, 23, 24, 26, 31, 32, 33, 35, 37, 38,
    39, 40, 41, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 60, 62, 63, 64,
    66, 67, 68, 70, 71, 77, 78, 79, 80, 83, 87, 88, 89, 90, 92, 93, 94, 95, 100,
    104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120,
    121, 122, 125, 128, 129, 130, 133, 134, 135, 137, 138, 140, 142, 143, 144,
    145, 147, 197, 198, 201, 202, 204, 205, 207, 208, 210, 215, 216, 217, 218,
    221, 223, 226, 228, 229, 230, 231, 233, 238, 240, 242, 245, 248, 249, 250,
    257, 258, 259, 260, 263, 267, 270, 272, 273, 275, 276, 277, 278, 279, 282,
    283, 284, 289, 291, 292, 296, 297, 306, 309, 310, 311, 313, 314, 316, 317,
    318, 319, 320, 327, 329, 330, 331, 332, 333, 335, 338, 342, 345, 346, 349,
    351, 354, 356, 357, 358, 360, 363, 364, 366, 372, 375, 378, 379, 380, 381,
    382, 386, 387, 388, 390, 391, 394, 399, 402, 405, 406, 408, 409, 411, 412,
    415, 417, 418, 427, 428, 429, 431, 432, 434, 435, 437, 448, 449, 451, 453,
    455, 457, 458, 460, 465, 466, 468, 471, 475, 478, 480, 482, 483, 485, 487,
    488, 490, 491, 492, 494, 495, 496, 500, 501, 502, 504, 506, 507, 508, 510,
    511, 514, 516, 521, 524, 526, 528, 529, 532, 533, 536, 541, 542, 543, 544,
    546, 547, 552, 556, 559, 561, 562, 563, 564, 566, 567, 569, 571, 587, 590,
    592, 593, 594, 597, 598, 599, 600, 601, 602, 605, 609, 611, 612, 614, 615,
    619, 622, 623, 625, 627, 628, 631, 633, 635, 636, 641, 643, 646, 648, 653,
    657, 661, 664, 665, 668, 671, 675, 678, 681, 684, 685, 690, 692, 693, 696,
    697, 707, 710, 712, 715, 721, 725, 731, 734, 737, 738, 742, 745, 750, 751,
    753, 754, 757, 760, 762, 770, 771, 773, 776, 777, 779, 786, 790, 791, 794,
    796, 800, 806, 812, 819, 821, 822, 826, 828, 829, 830, 838, 842, 847, 848,
    851, 853, 859, 863, 865, 867, 869, 874, 876, 878, 882, 883, 887, 892, 894,
    896, 906, 911, 921, 925, 928, 934, 938, 943, 944, 947, 950, 953, 957, 962,
    965, 967, 968, 973, 978, 980, 985, 987, 989, 990, 993, 994, 996, 997, 998,
    1000, 1004, 1005, 1006, 1007, 1011, 1014, 1017, 1019, 1023, 1025, 1027,
    1028, 1029, 1030, 1032, 1033, 1035, 1038, 1041, 1042, 1044, 1048, 1057,
    1062, 1069, 1073, 1074, 1075, 1077, 1078, 1079, 1084, 1085, 1088, 1090,
    1100, 1103, 1112, 1113, 1114, 1116, 1118, 1119, 1120, 1124, 1127, 1130,
    1136, 1137, 1138, 1139, 1141, 1142, 1143, 1151, 1153, 1155, 1156, 1157,
    1158, 1159, 1160, 1162, 1163, 1166, 1169, 1170, 1171, 1173, 1179, 1181,
    1182, 1184, 1185, 1188, 1193, 1194, 1196, 1199, 1204, 1205, 1206, 1209,
    1211, 1212, 1216, 1217, 1221, 1222, 1223, 1224, 1226, 1228, 1231, 1234,
    1237, 1239, 1240, 1241, 1243, 1244, 1245, 1248, 1249, 1250, 1251, 1253,
    1254, 1255, 1256, 1257, 1258, 1259, 1261, 1263, 1264, 1265, 1266, 1267,
    1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1282, 1284,
    1285, 1288, 1289
]

In [37]:

def carregar_vendas(caminho_arquivo, nome_aba):
    """Carrega o forecast e consolida vendas semanais por FILIAL + SKU."""
    print("Carregando vendas por SKU/PDV...")

    df = pd.read_excel(caminho_arquivo, sheet_name=nome_aba, engine="openpyxl")
    df.columns = df.columns.str.strip()

    df["FILIAL"] = df["FILIAL"].astype(str).str.strip()
    df["FILIAL"] = df["FILIAL"].str.extract(r"(\d+)").fillna("0")
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    df = df[df["FILIAL"].isin(lista_ra)].copy()


    print("Limpando colunas de origem para o SKU...")
    for coluna in ["PRODUTO", "COR", "TAMANHO"]:
        if coluna not in df.columns:
            raise KeyError(f"A coluna de origem '{coluna}' é necessária para criar o SKU, mas não foi encontrada.")
        df[coluna] = df[coluna].astype(str).str.strip()

    # 2. Criar a coluna 'SKU' concatenando as colunas limpas
    print("Criando coluna 'SKU' a partir de PRODUTO + COR + TAMANHO...")
    df["SKU"] = df["PRODUTO"] + df["COR"] + df["TAMANHO"]
    
    # 3. Padronizar a nova coluna SKU (maiúsculas e sem espaços)
    #    O .str.replace(' ', '') remove TODOS os espaços, o que é ótimo para um ID.
    df["SKU"] = df["SKU"].str.upper().str.replace(' ', '')


    for coluna in ["PRODUTO", "COR", "TAMANHO", "SKU"]:
        df[coluna] = df[coluna].astype(str).str.upper().str.strip()

    colunas_semana = [col for col in df.columns if col.startswith("202") and "_W" in col] # procura colunas que começam com "202" e contém "_W" ex: "2024_W39"
    rename_map = {}
    # Objetivo deste loop é renomear colunas de "2024_W39" para "W39_24"
    for coluna in colunas_semana:
        ano, semana = coluna.split("_W") # "2024_W39" → ("2024", "39")
        semana = semana.zfill(2) # garante que a semana tenha 2 dígitos, ex: "9" → "09"
        rename_map[coluna] = f"W{semana}_{ano[-2:]}" #cria o novo nome, ex: "W39_24"

    df = df.rename(columns=rename_map)
    #a função _ordernar_semana garante que 'W52_25 venha antes de 'W01_26'
    semanas = sorted(rename_map.values(), key=_ordenar_semana)
    if not semanas:
        raise ValueError("Nenhuma coluna de semana encontrada no arquivo de forecast.")

    colunas_util = ["FILIAL", "SKU", "PRODUTO", "COR", "TAMANHO"] + semanas
    df = df[colunas_util]

    vendas_numericas = df.groupby(["FILIAL", "SKU"], as_index=False)[semanas].sum()
    #agrupe por FILIAL e SKU, mantendo as colunas de texto (PRODUTO, COR, TAMANHO) usando first()
    vendas_texto = df.groupby(["FILIAL", "SKU"], as_index=False)[["PRODUTO", "COR", "TAMANHO"]].first()
    #junte as duas tabelas, a de texto e a de numeros
    vendas = vendas_texto.merge(vendas_numericas, on=["FILIAL", "SKU"], how="inner") 

    print(f"[OK] Vendas: {len(vendas)} SKU/PDV, {len(semanas)} semanas")
    return vendas, semanas

In [38]:
def _coluna_existente(df, candidatos, descricao): #Procura no DataFrame df por uma coluna que tenha um destes nomes: candidatos (lista de strings)
    for coluna in candidatos:
        if coluna in df.columns:
            return coluna
    raise ValueError(f"Não foi possível localizar a coluna de {descricao}: {df.columns.tolist()}")

In [39]:



def carregar_estoque(caminho_arquivo):
    """Carrega e consolida o estoque atual por FILIAL + SKU."""
    print("Carregando estoque atual por SKU/PDV...")

    df = pd.read_excel(caminho_arquivo, engine="openpyxl")
    df.columns = df.columns.str.strip() # Remover espaços extras
    coluna_artigo = _coluna_existente(df, ["Artigo", "ARTIGO", "artigo"], "artigo")
    coluna_tamanho = _coluna_existente(df, ["Tamanho", "TAMANHO", "tam"], "tamanho")
    print("Criando coluna 'SKU' a partir de 'Artigo' e 'Tamanho'...")
    df['SKU'] = (df[coluna_artigo].astype(str).str.strip() + 
                 df[coluna_tamanho].astype(str).str.strip())


    coluna_filial = _coluna_existente(df, ["FILIAL", "Filial", "Ponto Venda Cód", "Ponto Venda Cod", "PontoVda", "PontoVda Cod"], "filial") #"Procure no DataFrame df por uma coluna que tenha um destes nomes:se o nome da coluna for "Ponto venda cód", depois renomeia para "FILIAL"


    coluna_estoque = _coluna_existente(df, ["Estoque Total", "Estoque", "Estoque_Total"], "estoque") #"Procure no DataFrame df por uma coluna que tenha um destes nomes:

    df = df.rename(columns={coluna_filial: "FILIAL", coluna_estoque: "ESTOQUE_ATUAL"}) #Renomeia as colunas encontradas para "FILIAL" e "ESTOQUE_ATUAL"
    
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int) #Converte a coluna "FILIAL" para numérico, substitui erros por 0 e converte para inteiro
    df = df[df["FILIAL"].isin(lista_ra)].copy()
    df["SKU"] = df["SKU"].astype(str).str.upper().str.strip() #Converte a coluna "SKU" para string, maiúsculas e remove espaços extras
    df["ESTOQUE_ATUAL"] = pd.to_numeric(df["ESTOQUE_ATUAL"], errors="coerce").fillna(0) #Converte a coluna "ESTOQUE_ATUAL" para numérico, substitui erros por 0

    df = df.groupby(["FILIAL", "SKU"], as_index=False)["ESTOQUE_ATUAL"].sum() #Agrupa por "FILIAL" e "SKU", somando os valores de "ESTOQUE_ATUAL"
    print(f"[OK] Estoque consolidado: {len(df)} SKU/PDV")
    return df




In [40]:
def carregar_carteira(caminho_arquivo):
    """Carrega e consolida carteira de pedidos por FILIAL + SKU."""
    print("Carregando carteira por SKU/PDV...")

    df = pd.read_excel(caminho_arquivo, engine="openpyxl")
    df.columns = df.columns.str.strip()

    coluna_artigo = _coluna_existente(df, ["Artigo", "ARTIGO", "artigo"], "artigo")
    coluna_tamanho = _coluna_existente(df, ["Tamanho", "TAMANHO", "tam"], "tamanho")
    print("Criando coluna 'SKU' a partir de 'Artigo' e 'Tamanho'...")
    df['SKU'] = (df[coluna_artigo].astype(str).str.strip() + 
                 df[coluna_tamanho].astype(str).str.strip())

    coluna_filial = _coluna_existente(df, ["FILIAL", "Filial", "PontoVda", "PontoVda Cod", "Ponto Venda Cód", "Ponto Venda Cod"], "filial")
    coluna_pecas = _coluna_existente(df, ["Pecas", "Peças", "Pecas_total", "Pecas Totais"], "pecas")

    df = df.rename(columns={coluna_filial: "FILIAL", coluna_pecas: "CARTEIRA_TOTAL"})
    if "SKU" not in df.columns:
        raise ValueError("Coluna 'SKU' não encontrada na carteira.")
    
    df = df[df['FILIAL'].isin(lista_ra)].copy()
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    df["SKU"] = df["SKU"].astype(str).str.upper().str.strip()
    df["CARTEIRA_TOTAL"] = pd.to_numeric(df["CARTEIRA_TOTAL"], errors="coerce").fillna(0)

    df = df.groupby(["FILIAL", "SKU"], as_index=False)["CARTEIRA_TOTAL"].sum()
    print(f"[OK] Carteira consolidada: {len(df)} SKU/PDV")
    return df

### Lógica do Cálculo do Alvo de Estoque
O "alvo" representa o nível de estoque ideal para cada produto (SKU) em uma determinada semana. O objetivo é garantir que haja estoque suficiente para cobrir as vendas de um período futuro, evitando rupturas.

O cálculo é baseado em uma "janela deslizante" de vendas, cujo tamanho é definido pelo parâmetro SEMANAS_ALVO.

Como funciona na prática:

A cada semana da simulação, o código olha para a frente e soma as vendas projetadas para o número de semanas definido em SEMANAS_ALVO.
Essa soma se torna a meta de estoque (o "alvo") para a semana atual.
Exemplo: Se SEMANAS_ALVO = 10:

Para a Semana 42, o alvo será a soma das vendas projetadas da Semana 42 até a Semana 51.
Para a Semana 43, a janela desliza, e o alvo passa a ser a soma das vendas da Semana 43 até a Semana 52.
Esse processo se repete para cada semana do HORIZONTE_SEMANAS, criando uma meta de estoque dinâmica que se ajusta à projeção de vendas ao longo do tempo.

In [41]:
def calcular_alvos(df, todas_semanas, semana_inicio, horizonte_semanas, semanas_cobertura, fator_deflacao):
    """Calcula alvos de estoque com janela fixa e deflacao."""
    print("Calculando alvos de estoque...")

    if semana_inicio not in todas_semanas:
        raise ValueError(f"Semana inicial {semana_inicio} não encontrada no planejamento de vendas.")

    inicio_idx = todas_semanas.index(semana_inicio)

    if horizonte_semanas is None:
        semanas_simulacao = todas_semanas[inicio_idx:]
    else:
        fim_idx = min(len(todas_semanas), inicio_idx + horizonte_semanas)
        semanas_simulacao = todas_semanas[inicio_idx:fim_idx]

    if not semanas_simulacao:
        raise ValueError("Lista de semanas para simulação está vazia.")

    fator_deflacao = float(fator_deflacao)

    for semana in semanas_simulacao:
        idx_semana = todas_semanas.index(semana)
        fim_cobertura = min(len(todas_semanas), idx_semana + semanas_cobertura)
        semanas_para_somar = todas_semanas[idx_semana:fim_cobertura]
        if not semanas_para_somar:
            df[f"ALVO_{semana}"] = 0.0
            continue

        alvo_bruto = df[semanas_para_somar].sum(axis=1)
        df[f"ALVO_{semana}"] = alvo_bruto * fator_deflacao

    return df, semanas_simulacao


### Cenário 1: O seu exemplo (precisa repor)
Imagine um único produto na Semana 43:

estoque_corrente (no início da Sem 43): 100
vendas (projetadas para a Sem 43): 20
alvo (calculado para a Sem 43): 100
1. Cálculo da reposicao:

Primeiro, o código simula o estoque após a venda: 100 (estoque) - 20 (vendas) = 80
Depois, ele calcula o "buraco" a ser preenchido para atingir o alvo: 100 (alvo) - 80 (estoque pós-venda) = 20
Resultado: reposicao para a Semana 43 = 20 peças.


### 2. Cálculo do estoque_corrente para a próxima semana:

Agora, o código calcula o estoque real no final da semana, que será o estoque inicial da Semana 44.
Fórmula: Estoque Inicial - Vendas + Reposição
Cálculo: 100 - 20 + 20 = 100
Resultado: O estoque_corrente que será usado no início da Semana 44 é 100.

In [42]:
def simular_reposicao(df, semanas_simulacao):
    """Simula reposição semanal garantindo saldo mínimo zero."""
    print("Simulando reposição...")

    reposicoes = {} # armazenar as reposições calculadas de cada semana
    estoques = {} # armazenar os estoques finais de cada semana
    estoque_corrente = df["ESTOQUE_INICIAL"].astype(float).to_numpy(copy=True) # para manipulação numérica mais eficiente e segura 

    for semana in semanas_simulacao: #
        vendas = df[semana].astype(float).to_numpy(copy=True) # pegamos a coluna com a projeção de vendas daquela semana para cada SKU/PDV
        alvo = df[f"ALVO_{semana}"].astype(float).to_numpy(copy=True) # pegamos a coluna ALVO_X (estoque desejado) calculada previamente
        reposicao = np.maximum(0, alvo - (estoque_corrente - vendas)) #simula o saldo depois de as vendas ocorrerem (sem reposição).
        reposicao = reposicao * 1.20 
        reposicao = np.ceil(reposicao) #np.ceil(...): arredonda para cima, garantindo que a reposição seja um número inteiro
        estoque_corrente = np.maximum(0, estoque_corrente - vendas + reposicao)  #Subtraímos as vendas, somamos a reposição recém-calculada e novamente aplicamos np.maximum para garantir que o resultado nunca seja negativo

        reposicoes[f"REPOSICAO_{semana}"] = reposicao.astype(int) #Salvamos as reposições daquela semana (convertidas para inteiro) no dicionário reposicoes
        estoques[f"ESTOQUE_{semana}"] = estoque_corrente.copy()
    for coluna, valores in reposicoes.items(): # Quando fazemos reposicoes.items() recebemos pares (chave, valor)"ESTOQUE_W39_25": array([260, 175,  8, …]),
        df[coluna] = valores #cada chave vira o nome da coluna no DataFrame, e cada array vira o conteúdo daquela coluna
    for coluna, valores in estoques.items():
        df[coluna] = valores

    return df

In [43]:
def salvar_resultados(df, semanas_simulacao, pasta_saida):
    print("Salvando resultados...")

    dinamicas = {
        "vendas": list(semanas_simulacao), # lista das semanas simuladas
        "alvos": [f"ALVO_{s}" for s in semanas_simulacao], # lista das colunas de alvos correspondentes às semanas simuladas
        "reposicoes": [f"REPOSICAO_{s}" for s in semanas_simulacao],
        "estoques": [f"ESTOQUE_{s}" for s in semanas_simulacao],
    }

    for coluna in dinamicas["vendas"] + dinamicas["alvos"] + dinamicas["reposicoes"] + dinamicas["estoques"]:
        if coluna not in df.columns:
            df[coluna] = 0

    colunas_exportar = [
        "FILIAL",
        "SKU",
        "PRODUTO",
        "COR",
        "TAMANHO",
        "ESTOQUE_ATUAL",
        "CARTEIRA_TOTAL",
        "ESTOQUE_INICIAL",
    ]
    for grupo in dinamicas.values():
        colunas_exportar.extend(grupo)

    colunas_exportar = [col for col in colunas_exportar if col in df.columns]
    df_final = df[colunas_exportar].copy()

    pasta_saida.mkdir(exist_ok=True)
    destino = pasta_saida / "projecao_sku_pdv.xlsx"
    df_final.to_excel(destino, index=False)

    print(f"[OK] Resultados salvos em: {destino}")
    return df_final

In [44]:
def main():
    """Executa a simulação completa por SKU/PDV."""
    print("=" * 60)
    print("SIMULAÇÃO DE REPOSIÇÃO POR SKU/PDV")
    print("=" * 60)

    try:
        vendas, todas_semanas = carregar_vendas(ARQUIVO_VENDAS, ABA_VENDAS)
        estoque = carregar_estoque(ARQUIVO_ESTOQUE)
        carteira = carregar_carteira(ARQUIVO_CARTEIRA)

        print()
        print("Combinando dados de vendas, estoque e carteira...")
        dados = pd.merge(vendas, estoque, on=["FILIAL", "SKU"], how="inner")
        dados = pd.merge(dados, carteira, on=["FILIAL", "SKU"], how="left")

        dados["ESTOQUE_ATUAL"] = dados["ESTOQUE_ATUAL"].fillna(0)
        dados["CARTEIRA_TOTAL"] = dados["CARTEIRA_TOTAL"].fillna(0)
        dados["ESTOQUE_INICIAL"] = dados["ESTOQUE_ATUAL"] + dados["CARTEIRA_TOTAL"]

        dados, semanas_simulacao = calcular_alvos(dados, todas_semanas, SEMANA_INICIO, HORIZONTE_SEMANAS, SEMANAS_ALVO, FATOR_DEFALCACAO_ALVO)
        dados = simular_reposicao(dados, semanas_simulacao)
        resultado = salvar_resultados(dados, semanas_simulacao, SAIDA_DIR)

        print("=" * 60)
        print("SIMULAÇÃO CONCLUÍDA COM SUCESSO!")
        print(f"Período: {semanas_simulacao[0]} a {semanas_simulacao[-1]}")
        print(f"SKUs analisados: {len(resultado)}")
        print(f"Filiais: {resultado['FILIAL'].nunique()}")
        print("=" * 60)

    except Exception as err:
        print(f"ERRO: {err}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


SIMULAÇÃO DE REPOSIÇÃO POR SKU/PDV
Carregando vendas por SKU/PDV...
Limpando colunas de origem para o SKU...
Criando coluna 'SKU' a partir de PRODUTO + COR + TAMANHO...
[OK] Vendas: 356389 SKU/PDV, 45 semanas
Carregando estoque atual por SKU/PDV...
Criando coluna 'SKU' a partir de 'Artigo' e 'Tamanho'...
[OK] Estoque consolidado: 380693 SKU/PDV
Carregando carteira por SKU/PDV...
Criando coluna 'SKU' a partir de 'Artigo' e 'Tamanho'...
[OK] Carteira consolidada: 44488 SKU/PDV

Combinando dados de vendas, estoque e carteira...
Calculando alvos de estoque...
Simulando reposição...
Salvando resultados...
[OK] Resultados salvos em: outputs\projecao_sku_pdv.xlsx
SIMULAÇÃO CONCLUÍDA COM SUCESSO!
Período: W42_25 a W13_26
SKUs analisados: 329755
Filiais: 535
